In [1]:
import torch
from tqdm import tqdm
import numpy as np
import sys
sys.path.append('../')
import argparse
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from data import load_dataset
from utils import get_llama_activations_bau

2025-01-04 16:47:14.439579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736009234.457515    4539 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736009234.462972    4539 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
HF_NAMES = {
    'llama3.1_8B': 'meta-llama/Llama-3.1-8B'
}

model_name = HF_NAMES['llama3.1_8B']
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
device = 'cuda'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
data_v = load_dataset('./data/verifiable.csv')
data_nv = load_dataset('./data/non-verifiable.csv')
data_v_100 = data_v[:100]
data_nv_100 = data_nv[:100]
data_all = data_v_100 + data_nv_100
print(len(data_all))


900it [00:00, 27906.01it/s]

200


In [4]:
label_v = np.ones(len(data_v_100))
label_nv = np.zeros(len(data_nv_100))
label_all = np.concatenate((label_v, label_nv))
print(len(label_all))

200


In [5]:
all_layer_wise_activations = []
all_head_wise_activations = []

In [6]:
print('Getting activations')
for i, inp in enumerate(tqdm(data_all)):
    torch.cuda.empty_cache()
    prompt = tokenizer(inp, return_tensors='pt').input_ids
    layer_wise_activations, head_wise_activations, _ = get_llama_activations_bau(model, prompt, device)
    all_layer_wise_activations.append(layer_wise_activations[:,-1,:].copy())
    all_head_wise_activations.append(head_wise_activations[:,-1,:].copy())

Getting activations



00%|██████████| 200/200 [04:04<00:00,  1.22s/it]

In [11]:
print('Saving labels')
np.save(f'./features/llama3.1_8B_uncertainty_labels.npy', label_all)

Saving labels


In [12]:
print('Saving layer wise activations')
np.save(f'./features/llama3.1_8B_uncertainty_layer_wise.npy', all_layer_wise_activations)

Saving layer wise activations


In [13]:
print('Saving head wise activations')
np.save(f'./features/llama3.1_8B_uncertainty_head_wise.npy', all_head_wise_activations)

Saving head wise activations
